In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
if not os.path.exists("model_data/yolo.h5"):
  # 下載 yolov3 的網路權重，並且把權重轉換為 keras 能夠讀取的格式
  print("Model doesn't exist, downloading...")
  os.system("wget https://pjreddie.com/media/files/yolov3.weights")
  print("Converting yolov3.weights to yolo.h5...")
  os.system("python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5")
else:
  print("Model exist")

Model exist


In [3]:
# 把 annotation 轉換訓練時需要的資料形態
import xml.etree.ElementTree as ET # 載入能夠 Parser xml 文件的 library
def convert_annotation(class_name, image_id):
    file_path = f'dataset/{class_name}/annotations/{image_id}.xml'
    img_path = f'dataset/{class_name}/images/{image_id}.jpg'
    in_file = open(file_path)
    
    tree = ET.parse(in_file)
    root = tree.getroot()
    all_txt = ''
    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text))
        txt = "./"+img_path + " " + ",".join([str(a) for a in b]) + ',' + str(cls_id)
        txt2 = ",".join([str(a) for a in b]) + ',' + str(cls_id)
        if len(all_txt)>0:
            all_txt+=' '+txt2
        else:
            all_txt+=txt
    return all_txt

In [4]:
import time
classes = ['raccoon', 'kangaroo', 'kangaroo']
dirs = ['raccoon', 'kangaroo','k']
trian_file = 'D50_train0308.txt'
all_txt = ''
if not os.path.exists(trian_file): 
    list_file = open(trian_file, 'a')
    for i in range(len(classes)):
        for files in os.walk(f'dataset/{dirs[i]}/images'):
            all_files = files[2]
            for filename in all_files:
                filename = filename.split('.')[0]
                all_txt += f'{convert_annotation(dirs[i], filename)}\n'
    list_file.write(all_txt)
else:
    print("D50_train0308.txt exist")

D50_train0308.txt exist


In [5]:
if not os.path.exists("model_data/yolo_weights.h5"):
  print("Converting pretrained YOLOv3 weights for training")
  os.system("python convert.py -w yolov3.cfg yolov3.weights model_data/yolo_weights.h5") 
else:
  print("Pretrained weights exists")

Pretrained weights exists


In [6]:
# 將 train.py 所需要的套件載入
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data

from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [7]:
from train import get_classes, get_anchors, create_model, create_tiny_model, data_generator, data_generator_wrapper
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
CUDA_VISIBLE_DEVICES = 0

In [8]:
annotation_path = 'D50_train0308.txt' # 轉換好格式的標註檔案
log_dir = 'logs/0038/' # 訓練好的模型儲存的路徑
classes_path = 'model_data/D50_classes.txt'
anchors_path = 'model_data/yolo_anchors.txt'
class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)

In [9]:
# 分為 training 以及 validation
val_split = 0.2
with open(annotation_path) as f:
    lines = f.readlines()
np.random.seed(10101)
np.random.shuffle(lines)
num_val = int(len(lines)*val_split)
num_train = len(lines) - num_val
(num_val, num_train)

(80, 320)

In [10]:
input_shape = (416,416) # multiple of 32, hw

is_tiny_version = len(anchors)==6 # default setting
if is_tiny_version:
    model = create_tiny_model(input_shape, anchors, num_classes,
        freeze_body=2, weights_path='model_data/tiny_yolo_weights.h5')
else:
    model = create_model(input_shape, anchors, num_classes,
        freeze_body=2, weights_path='model_data/yolo_weights.h5') # make sure you know what you freeze

logging = TensorBoard(log_dir=log_dir)
checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

Create YOLOv3 model with 9 anchors and 2 classes.
Load weights model_data/yolo_weights.h5.
Freeze the first 249 layers of total 252 layers.


In [11]:
# Train with frozen layers first, to get a stable loss.
# Adjust num epochs to your dataset. This step is enough to obtain a not bad model.
# 一開始先 freeze YOLO 除了 output layer 以外的 darknet53 backbone 來 train
if True:
    model.compile(optimizer=Adam(lr=1e-3), loss={
        # use custom yolo_loss Lambda layer.
        'yolo_loss': lambda y_true, y_pred: y_pred})

    batch_size = 2
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
    # 模型利用 generator 產生的資料做訓練，強烈建議大家去閱讀及理解 data_generator_wrapper 在 train.py 中的實現
    model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=50,
            initial_epoch=0,
            callbacks=[logging, checkpoint])
    model.save_weights(log_dir + 'trained_weights_stage_1.h5')

# Unfreeze and continue training, to fine-tune.
# Train longer if the result is not good.
if True:
    # 把所有 layer 都改為 trainable
    for i in range(len(model.layers)):
        model.layers[i].trainable = True
    model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
    print('Unfreeze all of the layers.')

    batch_size = 2 # note that more GPU memory is required after unfreezing the body
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
    model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
        steps_per_epoch=max(1, num_train//batch_size),
        validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
        validation_steps=max(1, num_val//batch_size),
        epochs=100,
        initial_epoch=50,
        callbacks=[logging, checkpoint, reduce_lr, early_stopping])
    model.save_weights(log_dir + 'trained_weights_final.h5')

Train on 320 samples, val on 80 samples, with batch size 2.
Epoch 1/50
160/160 [==============================] - 42s 263ms/step - loss: 636.5700 - val_loss: 71.3538
Epoch 2/50
160/160 [==============================] - 38s 236ms/step - loss: 58.7468 - val_loss: 39.3061
Epoch 3/50
160/160 [==============================] - 38s 237ms/step - loss: 38.1923 - val_loss: 30.4636
Epoch 4/50
160/160 [==============================] - 37s 234ms/step - loss: 30.9946 - val_loss: 26.4833
Epoch 5/50
160/160 [==============================] - 37s 231ms/step - loss: 27.3244 - val_loss: 24.1052
Epoch 6/50
160/160 [==============================] - 37s 232ms/step - loss: 24.8450 - val_loss: 22.1191
Epoch 7/50
160/160 [==============================] - 37s 232ms/step - loss: 23.4803 - val_loss: 21.1333
Epoch 8/50
160/160 [==============================] - 37s 232ms/step - loss: 22.3561 - val_loss: 20.5589
Epoch 9/50
160/160 [==============================] - 37s 233ms/step - loss: 21.9822 - val_loss: 19

160/160 [==============================] - 112s 697ms/step - loss: 14.0378 - val_loss: 14.3770

Epoch 00073: ReduceLROnPlateau reducing learning rate to 9.999999939225292e-10.
Epoch 74/100
160/160 [==============================] - 111s 691ms/step - loss: 14.0390 - val_loss: 14.9993
Epoch 00074: early stopping
